In [131]:
import numpy as np
import pandas as pd

In [133]:
df=pd.read_csv("titanic.csv")

In [135]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [137]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [139]:
df=df.drop(columns=["PassengerId","Name","Ticket","Cabin"])

In [141]:
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S
2,0,2,male,62.0,0,0,9.6875,Q


In [143]:
df=pd.get_dummies(df,columns=["Sex","Embarked"])

In [145]:
df.head(3)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,34.5,0,0,7.8292,False,True,False,True,False
1,1,3,47.0,1,0,7.0000,True,False,False,False,True
2,0,2,62.0,0,0,9.6875,False,True,False,True,False


In [147]:
from sklearn.impute import SimpleImputer
si=SimpleImputer()
df["Age"]=si.fit_transform(df[["Age"]])
df["Fare"]=si.fit_transform(df[["Fare"]])

In [149]:
df.isnull().sum()

Survived      0
Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [151]:
df["Survived"].value_counts()

Survived
0    266
1    152
Name: count, dtype: int64

In [153]:
x=df.drop(columns="Survived")
y=df["Survived"]

In [155]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [157]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
pipe=Pipeline([
    ("scaler",StandardScaler()),
    ("pca",PCA(n_components=2)),
    ("classifier",RandomForestClassifier(n_estimators=100,random_state=42))
])

In [159]:
pipe

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [161]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [165]:
y_pred=pipe.predict(x_test)

In [167]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

1.0

In [179]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Lasso
parameterdict={"alpha":[0.1,1.0,10.0,100.0]}
lasso=Lasso()
random_search=RandomizedSearchCV( lasso, parameterdict, n_iter=2, cv=6)
random_search.fit(x_train,y_train)

RandomizedSearchCV(cv=6, estimator=Lasso(), n_iter=2,
                   param_distributions={'alpha': [0.1, 1.0, 10.0, 100.0]})

In [181]:
y_pred_new=random_search.predict(x_test)

In [183]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred_new)

0.7957068751293077